In [ ]:
from flask import Flask, render_template, jsonify, Response,request, session, current_app,redirect, url_for
import numpy as np
import pandas as pd
import base64
import io 
import base64
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from qiskit.primitives import Sampler
from qiskit import Aer
from qiskit import transpile, execute
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import SPSA,COBYLA
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.algorithms.classifiers import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.exceptions import QiskitMachineLearningError
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit.visualization import circuit_drawer
import time



app = Flask(__name__)
objective_func_vals = []
@app.route('/',methods=['GET', 'POST'])
def index():
    
 
   
    if request.method == 'POST':
        
        model = request.form['model']
        dataset = request.form['dataset']
        feature_map_type = request.form['featuremap']
        optimizer_type = request.form['optimizer']
        split = request.form['split']
        pca_no = request.form['pca']
        pca_no = int(pca_no)
        entang = request.form['entang']
       
        
        
        
        X,y= load_dataset(dataset)
        
        feature_no = X.shape[1]
        
        X = pca(pca_no, X)
        
        test_size = split_ratio(split)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        
        X_train_scaled, X_test_scaled = preprocess_data(X_train, X_test)
        
        
       
    
        feature_map, ansatz, base64_feature_map_image, base64_ansatz_image = create_feature_map(feature_map_type, pca_no, entang)
        
        optimizer = create_optimizer(optimizer_type)    
        
        import threading
        def train(model,optimizer,feature_map,ansatz,X_train_scaled, X_test_scaled, y_train, y_test):
            
                load_model(model,optimizer,feature_map,ansatz,X_train_scaled, X_test_scaled, y_train, y_test)
            
            
            
        train_thread = threading.Thread(target=train, args=(model,optimizer,feature_map,ansatz,X_train_scaled, X_test_scaled, y_train, y_test))
        train_thread.start()
        
        return render_template('index.html',fmap=base64_feature_map_image, ans = base64_ansatz_image, dname = dataset, pcano=pca_no,
                               ent = entang, opt = optimizer, mod = model, f_no = feature_no)
    
    
       
          
       
    #return redirect(url_for('index'))
    return render_template('index.html')
    
def load_dataset(dataset):
    if dataset == 'iris':
        iris = datasets.load_iris()
        X = iris.data
        y = iris.target
        return X, y
    if dataset == 'diabetes':
        diabetes = datasets.load_diabetes()
        X = diabetes.data
        y = diabetes.target
        return X, y
    if dataset == 'wine':
        wine = datasets.load_wine()
        X = wine.data
        y = wine.target
        return X, y
    if dataset == 'cancer':
        cancer = datasets.load_breast_cancer()
        X = cancer.data
        y = cancer.target
        return X, y
    
def split_ratio(split):
    if split == '80/20':
        return 0.2
    else :
        return 0.3

def preprocess_data(X_train, X_test):
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

def draw_and_encode_circuit(circuit):
    image_stream = io.BytesIO()
    circuit_drawer(circuit, output="mpl", fold=20, filename=image_stream)
    image_stream.seek(0)
    base64_image = base64.b64encode(image_stream.getvalue()).decode('utf-8')
    return base64_image

def create_feature_map(feature_map_type, fdimension, entang):
    if feature_map_type == 'zz':
        # Create the ZZFeatureMap and draw its circuit
        feature_map = ZZFeatureMap(feature_dimension=fdimension, reps=1, entanglement=entang)
        base64_feature_map_image = draw_and_encode_circuit(feature_map.decompose())

        # Create the RealAmplitudes ansatz and draw its circuit
        ansatz = RealAmplitudes(num_qubits=fdimension, reps=3)
        base64_ansatz_image = draw_and_encode_circuit(ansatz.decompose())

        return feature_map,ansatz, base64_feature_map_image, base64_ansatz_image
    
    elif feature_map_type == 'z':
        # Create the ZFeatureMap and draw its circuit
        feature_map = ZFeatureMap(feature_dimension=fdimension, reps=1, entanglement=entang)
        base64_feature_map_image = draw_and_encode_circuit(feature_map.decompose())

        # Create the RealAmplitudes ansatz and draw its circuit
        ansatz = RealAmplitudes(num_qubits=fdimension, reps=3)
        base64_ansatz_image = draw_and_encode_circuit(ansatz.decompose())

        return feature_map,ansatz,base64_feature_map_image, base64_ansatz_image
    
    elif feature_map_type == 'pauli':
        # Create the ZZFeatureMap and draw its circuit
        feature_map = PauliFeatureMap(feature_dimension=fdimension, reps=1, entanglement=entang)
        base64_feature_map_image = draw_and_encode_circuit(feature_map.decompose())

        # Create the RealAmplitudes ansatz and draw its circuit
        ansatz = RealAmplitudes(num_qubits=fdimension, reps=3)
        base64_ansatz_image = draw_and_encode_circuit(ansatz.decompose())

        return feature_map,ansatz,base64_feature_map_image, base64_ansatz_image

def create_optimizer(optimizer_type):
    if optimizer_type == 'spsa':
        return SPSA(maxiter=100)
    if optimizer_type == 'coby':
        return COBYLA(maxiter=100)
    if optimizer_type == 'adam':
        return COBYLA(maxiter=100)
    
def pca(pca_no,X):
    pca = decomposition.PCA(n_components=pca_no)
    pca.fit(X)
    X = pca.transform(X)
    return X   
    
    
     
#model,optimizer,feature_map,ansatz, X_train_scaled, X_test_scaled, y_train, y_test

def load_model(model,optimizer,feature_map,ansatz,X_train_scaled, X_test_scaled, y_train, y_test):
   
    if model == "VQC/MLP":
        c_metrics = mlp_classifier(X_train_scaled, X_test_scaled, y_train, y_test)
        q_metrics = vqc_classifier(optimizer,feature_map,ansatz, X_train_scaled, X_test_scaled, y_train, y_test)        
        barchart_mlp_vqc(model,c_metrics,q_metrics)
        
    
    if model == "QSVM/SVM":
        c_metrics = svm_classifier(X_train_scaled, X_test_scaled, y_train, y_test)
        q_metrics = qsvm_classifier(feature_map,X_train_scaled, X_test_scaled, y_train, y_test)
        barchart_mlp_vqc(model,c_metrics,q_metrics)
        
    
    if model == "QSVC/SVC":
        c_metrics = svc_classifier(X_train_scaled, X_test_scaled, y_train, y_test)
        q_metrics = qsvc_classifier(feature_map,X_train_scaled, X_test_scaled, y_train, y_test)
        barchart_mlp_vqc(model,c_metrics,q_metrics)
    
    if model == "VQC/LR":
        c_metrics = lr_classifier(X_train_scaled, X_test_scaled, y_train, y_test)
        q_metrics = vqc_classifier(optimizer,feature_map,ansatz,X_train_scaled, X_test_scaled, y_train, y_test)
        barchart_mlp_vqc(model,c_metrics,q_metrics)

#=============================== MLP / VQC CLASSIFIER==========================

def mlp_classifier(X_train_scaled, X_test_scaled, y_train, y_test):
    
    # Create an instance of the MLP classifier
    mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=5000, random_state=42)
    # Train the MLP classifier
    mlp.fit(X_train_scaled, y_train)
    # Make predictions with MLP classifier
    y_pred_mlp = mlp.predict(X_test_scaled)
    # Calculate accuracy, F1 score, precision, and recall for MLP classifier
    accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
    f1_mlp = f1_score(y_test, y_pred_mlp, average='weighted')
    precision_mlp = precision_score(y_test, y_pred_mlp, average='weighted')
    recall_mlp = recall_score(y_test, y_pred_mlp, average='weighted') 
    
    return accuracy_mlp, f1_mlp, precision_mlp, recall_mlp





def callback_graph(weights, obj_func_eval):
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.savefig('static/graph.jpg')  # Save the graph as an image
    plt.close()

def vqc_classifier(optimizer,feature_map,ansatz, X_train_scaled, X_test_scaled, y_train, y_test):
    
    optimizer = optimizer
    sampler = Sampler()
    vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    callback=callback_graph,

    )

    #import threading
    #def run_callback():
    vqc.fit(X_train_scaled, y_train)

    #train_thread = threading.Thread(target=run_callback)
    #train_thread.start()
    
    # Wait for the training thread to finish
    #train_thread.join()


    # Make predictions with VQC
    y_pred_vqc = vqc.predict(X_test_scaled)
    # Calculate accuracy, F1 score, precision, and recall for VQC
    accuracy_vqc = accuracy_score(y_test, y_pred_vqc)
    f1_vqc = f1_score(y_test, y_pred_vqc, average='weighted')
    precision_vqc = precision_score(y_test, y_pred_vqc, average='weighted')
    recall_vqc = recall_score(y_test, y_pred_vqc, average='weighted')
    print(accuracy_vqc)
    print(f1_vqc)
    print(precision_vqc)
    print( recall_vqc)
    return accuracy_vqc ,f1_vqc,precision_vqc, recall_vqc

def barchart_mlp_vqc(model, c_metrics, q_metrics):
    metrics = {}
    quantum, classical = model.split('/')
    # Bar chart
    metrics['c'] = c_metrics
    metrics['q'] = q_metrics
    accuracy_c, f1_c, precision_c, recall_c = metrics['c']
    accuracy_q, f1_q, precision_q, recall_q = metrics['q']

    labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    c_scores = [accuracy_c, precision_c, recall_c, f1_c]
    q_scores = [accuracy_q, precision_q, recall_q, f1_q]

    x = np.arange(len(labels))
    width = 0.35

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, c_scores, width, label=classical)
    rects2 = ax.bar(x + width/2, q_scores, width, label=quantum)

    ax.set_ylabel('Scores')
    ax.set_title(f'Comparison of Metrics: {classical} vs {quantum}')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    fig.tight_layout()

    # Save the plot as a JPG image in the static folder
    plt.savefig('static/barchart.jpg')

    # Close the plot to free up resources
    plt.close()
 
#================================= QSVM/SVM======================================================

def svm_classifier(X_train_scaled, X_test_scaled, y_train, y_test):
    
    svm = SVC(kernel='linear')
    svm.fit(X_train_scaled, y_train)
    svm_pred = svm.predict(X_test_scaled)
    
    svm_accuracy = accuracy_score(y_test, svm_pred)
    svm_precision = precision_score(y_test, svm_pred, average='macro')
    svm_recall = recall_score(y_test, svm_pred, average='macro')
    svm_f1 = f1_score(y_test, svm_pred, average='macro')
    print(svm_accuracy)
    print(svm_f1)
    print(svm_precision)
    print( svm_recall)
    return svm_accuracy,svm_precision, svm_recall,svm_f1
    
    
def qsvm_classifier(feature_map, X_train_scaled, X_test_scaled, y_train, y_test):
    
    qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=Aer.get_backend('qasm_simulator'))
    qsvm = QSVC(quantum_kernel=qkernel)
    #start = time.time()
    #epochs = 4
    #for _ in tqdm(range(epochs), desc="Training Progress"):
    qsvm.fit(X_train_scaled, y_train)
    #elapsed = time.time() - start
    # Predict labels for the test set
    y_pred = qsvm.predict(X_test_scaled)
    #cm = confusion_matrix(y_test, y_pred)
    qsvm_accuracy = np.sum(y_pred == y_test) / len(y_test)
    qsvm_f1 = f1_score(y_test, y_pred, average='weighted')
    qsvm_precision = precision_score(y_test, y_pred, average='weighted')
    qsvm_recall = recall_score(y_test, y_pred, average='weighted')
    print(qsvm_accuracy)
    print(qsvm_f1)
    print(qsvm_precision)
    print( qsvm_recall)
    return qsvm_accuracy,qsvm_f1,qsvm_precision,qsvm_recall
  

#================================QSVC/SVC======================================================

def svc_classifier(X_train_scaled, X_test_scaled, y_train, y_test):
    svc = SVC()
    svc.fit(X_train_scaled, y_train) 
    svc_pred = svc.predict(X_test_scaled)
    
    svc_accuracy = accuracy_score(y_test, svc_pred)
    svc_precision = precision_score(y_test, svc_pred, average='macro')
    svc_recall = recall_score(y_test, svc_pred, average='macro')
    svc_f1 = f1_score(y_test, svc_pred, average='macro')
    return svc_accuracy,svc_precision, svc_recall,svc_f1

def qsvc_classifier(feature_map, X_train_scaled, X_test_scaled, y_train, y_test):
    sampler = Sampler()
    fidelity = ComputeUncompute(sampler=sampler)
    kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    qsvc = QSVC(quantum_kernel=kernel)
    start = time.time()
    qsvc.fit(X_train_scaled, y_train)
    elapsed = time.time() - start
    y_pred = qsvc.predict(X_test_scaled)
    #cm = confusion_matrix(y_test, y_pred)
    qsvc_accuracy = np.sum(y_pred == y_test) / len(y_test)
    qsvc_f1 = f1_score(y_test, y_pred, average='weighted')
    qsvc_precision = precision_score(y_test, y_pred, average='weighted')
    qsvc_recall = recall_score(y_test, y_pred, average='weighted')    
    return qsvc_accuracy,qsvc_f1,qsvc_precision,qsvc_recall


#================================VQC/LR======================================================


    
def lr_classifier(X_train_scaled, X_test_scaled, y_train, y_test):
    # Create a logistic regression CL_model_LR
    lr = LogisticRegression()
    # Train the CL_model_LR
    lr.fit(X_train_scaled,  y_train)
    # Make predictions on the test set
    y_pred = lr.predict(X_test_scaled)
    lr_accuracy = accuracy_score(y_test, y_pred)
    lr_precision = precision_score(y_test, y_pred, average='macro')
    lr_recall = recall_score(y_test, y_pred, average='macro')
    lr_f1 = f1_score(y_test, y_pred, average='macro')
    return lr_accuracy,lr_precision, lr_recall,lr_f1



def ansatz_cir(N):
    image_stream = io.BytesIO()
    ansatz = RealAmplitudes(num_qubits=N, reps=3)
    ansatz.decompose().draw(output="mpl", fold=20,filename=image_stream )
    image_stream.seek(0)
    base64_image = base64.b64encode(image_stream.getvalue()).decode('utf-8')
    return base64_image

    
@app.route('/get_graph_data')
def get_graph_data():
    # Return the current timestamp as a unique parameter to prevent caching
    timestamp = int(time.time() * 1000)
    return jsonify({'graph_src': f'static/graph.jpg?{timestamp}'})

@app.route('/get_barchart')
def get_barchart():
    # Return the current timestamp as a unique parameter to prevent caching
    timestamp = int(time.time() * 3000)
    return jsonify({'bar_src': f'static/barchart.jpg?{timestamp}'})


@app.route('/quantum_job')
def quantum_job_route():
    if quantum_job.training_complete:
        quantum_job.run_quantum_job() 
        quantum_job.training_complete = False
        
        return jsonify({'result': 'success', 'progress': 0})
    else:
        return jsonify({'result': 'training_in_progress', 'progress': quantum_job.progress})

@app.route('/get_hist')
def get_hist():
     # Return the current timestamp as a unique parameter to prevent caching
    timestamp = int(time.time() * 4000)
    return jsonify({'hist_src': f'static/histogram.jpg?{timestamp}'})

    


    

if __name__ == '__main__':
    app.run(port=5889, debug=False, use_reloader=False)   
    
#def connect_to_backend(backend_name):
   # if backend_name == 'local':
      #  return Aer.get_backend('statevector_simulator')
   # elif backend_name == 'ionq':
      #  provider = AzureQuantumProvider(
          #  resource_id='/subscriptions/your_subscription_id/resourceGroups/your_resource_group/providers/Microsoft.Quantum/Workspaces/your_workspace_id',
           # location='your_location'
      #  )
       # return provider.get_backend('ionq.qpu.h1')
    # Add more backends here if needed


    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5889
Press CTRL+C to quit
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /static/graph.jpg HTTP/1.1" 304 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /static/barchart.jpg HTTP/1.1" 304 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /static/histogram.jpg HTTP/1.1" 304 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /get_barchart HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /static/graph.jpg?1690380883536 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /static/histogram.jpg?6761523534193 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /get_barchart HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:14:43] "GET /static/barchart.jpg?5071142650660 HTTP/1.1" 200 -
127.0.0.1 - - [26

127.0.0.1 - - [26/Jul/2023 15:15:10] "GET /static/barchart.jpg?5071142732429 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:11] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:11] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:11] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:11] "GET /static/histogram.jpg?6761523647125 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:11] "GET /static/histogram.jpg?6761523647260 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:11] "GET /static/graph.jpg?1690380911830 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:13] "GET /get_barchart HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:13] "GET /static/barchart.jpg?5071142741442 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:13] "GET /get_barchart HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:13] "GET /static/barchart.jpg?5071142741487 HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2023 15:15:13] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [